In [1]:
%env SPARK_VERSION=3.2.0

env: SPARK_VERSION=3.2.0


In [2]:
import pydeequ
from pyspark.sql import SparkSession, Row


spark = (
    SparkSession.builder
    .appName("pydeequ")
    .config("spark.jars", "/home/jovyan/mssql-jdbc-12.4.0.jre11.jar, /home/jovyan/deequ-2.0.3-spark-3.3.jar")
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate()
)


In [3]:
url = "jdbc:sqlserver://host.docker.internal:1433;databaseName=AdventureWorks2012;user=testlogin;password=testPa$$24;encrypt=true;trustServerCertificate=true" 

table = "Production.Product"
driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

In [4]:
# connect to DB with Spark using JDBC connection to read the data
products_df = (
    spark.read
    .format("jdbc")
    .option("driver",driver)
    .option("url", url)
    .option("dbtable", table)
    .load()
)

products_df.show()

+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-

### Data Analyzers section

In [5]:
from pydeequ.analyzers import *


analysisResult = (
    AnalysisRunner(spark)
    .onData(products_df)
    .addAnalyzer(Size())
    .addAnalyzer(Completeness("Name"))
    .addAnalyzer(Mean("StandardCost"))
    .addAnalyzer(Maximum("Weight"))
    .addAnalyzer(Minimum("Weight"))
    .addAnalyzer(CountDistinct("Color"))
    .addAnalyzer(Compliance("Color_specified", "Color IS NOT NULL"))
    .run()
)

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-------+---------------+-------------+------------------+
| entity|       instance|         name|             value|
+-------+---------------+-------------+------------------+
| Column|Color_specified|   Compliance|0.5079365079365079|
| Column|         Weight|      Maximum|            1050.0|
| Column|         Weight|      Minimum|              2.12|
| Column|   StandardCost|         Mean|258.60296130952383|
| Column|           Name| Completeness|               1.0|
| Column|          Color|CountDistinct|               9.0|
|Dataset|              *|         Size|             504.0|
+-------+---------------+-------------+------------------+



### Data profiling section

##### Data profiling and Constraint suggestion sections were done using pydeequ documentation: https://pypi.org/project/pydeequ/1.0.1/. 
##### Error were resolved after switching from Spark v.3.5.0 to v.3.2.0

In [6]:
from pydeequ.profiles import *

result = ColumnProfilerRunner(spark) \
    .onData(products_df) \
    .run()

for col, profile in result.profiles.items():
    print(profile)

StandardProfiles for column: ModifiedDate: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 2,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": null
}
NumericProfiles for column: ReorderPoint: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 6,
    "dataType": "Integral",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": [
        [
            "45",
            5,
            0.00992063492063492
        ],
        [
            "750",
            156,
            0.30952380952380953
        ],
        [
            "375",
            167,
            0.33134920634920634
        ],
        [
            "75",
            97,
            0.19246031746031747
        ],
        [
            "600",
            25,
            0.0496031746031746
        ],
        [
            "3",
            54,
            0.10714285714285714
        ]
    ],
    "kll": "None",
    "mean": 401.36309523809

### Constraint Suggestions section

In [7]:
from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(products_df) \
             .addConstraintRule(DEFAULT()) \
             .run()

# Constraint Suggestions in JSON format
print(suggestionResult)

{'constraint_suggestions': [{'constraint_name': 'CompletenessConstraint(Completeness(ModifiedDate,None))', 'column_name': 'ModifiedDate', 'current_value': 'Completeness: 1.0', 'description': "'ModifiedDate' is not null", 'suggesting_rule': 'CompleteIfCompleteRule()', 'rule_description': 'If a column is complete in the sample, we suggest a NOT NULL constraint', 'code_for_constraint': '.isComplete("ModifiedDate")'}, {'constraint_name': "ComplianceConstraint(Compliance('ReorderPoint' has value range '375', '750', '75', '3', '600', '45',`ReorderPoint` IN ('375', '750', '75', '3', '600', '45'),None))", 'column_name': 'ReorderPoint', 'current_value': 'Compliance: 1', 'description': "'ReorderPoint' has value range '375', '750', '75', '3', '600', '45'", 'suggesting_rule': 'CategoricalRangeRule(com.amazon.deequ.suggestions.rules.CategoricalRangeRule$$$Lambda$3605/0x000000084155a840@18c19d99)', 'rule_description': 'If we see a categorical range for a column, we suggest an IS IN (...) constraint'

### Constraint Verification section

In [8]:
from pydeequ.checks import *
from pydeequ.verification import *


check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = (
    VerificationSuite(spark)
    .onData(products_df)
    .addCheck(
        check.isUnique("ProductID")
        .isNonNegative("ProductID")
        .isComplete("ReorderPoint")
        .isNonNegative("ReorderPoint")
        .hasCompleteness("ProductModelID", lambda x: x >= 0.54, "It should be above 0.54!")
        .hasMin("StandardCost", lambda x: x == 0)
        .isNonNegative("Weight")
        .isUnique("ProductID")
        .isContainedIn("Style", ["U ", "W ", "M "])
        .isContainedIn("Color", ["Black", "Silver", "Red", "Yellow", "Blue", "Multi", "Silver/Black", "White", "Grey", "Brown"])
        .isComplete("ModifiedDate")
    )
    .run()
)

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

Python Callback server started!
+------------+-----------+------------+--------------------+-----------------+------------------+
|       check|check_level|check_status|          constraint|constraint_status|constraint_message|
+------------+-----------+------------+--------------------+-----------------+------------------+
|Review Check|    Warning|     Success|UniquenessConstra...|          Success|                  |
|Review Check|    Warning|     Success|ComplianceConstra...|          Success|                  |
|Review Check|    Warning|     Success|CompletenessConst...|          Success|                  |
|Review Check|    Warning|     Success|ComplianceConstra...|          Success|                  |
|Review Check|    Warning|     Success|CompletenessConst...|          Success|                  |
|Review Check|    Warning|     Success|MinimumConstraint...|          Success|                  |
|Review Check|    Warning|     Success|ComplianceConstra...|          Success|        

##### Report

In [9]:

from IPython.display import display, HTML

checkResult_df = checkResult_df.toPandas()

html_report = checkResult_df.to_html()
display(HTML(html_report))

,check,check_level,check_status,constraint,constraint_status,constraint_message
0,Review Check,Warning,Success,"UniquenessConstraint(Uniqueness(List(ProductID),None))",Success,
1,Review Check,Warning,Success,"ComplianceConstraint(Compliance(ProductID is non-negative,COALESCE(CAST(ProductID AS DECIMAL(20,10)), 0.0) >= 0,None))",Success,
2,Review Check,Warning,Success,"CompletenessConstraint(Completeness(ReorderPoint,None))",Success,
3,Review Check,Warning,Success,"ComplianceConstraint(Compliance(ReorderPoint is non-negative,COALESCE(CAST(ReorderPoint AS DECIMAL(20,10)), 0.0) >= 0,None))",Success,
4,Review Check,Warning,Success,"CompletenessConstraint(Completeness(ProductModelID,None))",Success,
5,Review Check,Warning,Success,"MinimumConstraint(Minimum(StandardCost,None))",Success,
6,Review Check,Warning,Success,"ComplianceConstraint(Compliance(Weight is non-negative,COALESCE(CAST(Weight AS DECIMAL(20,10)), 0.0) >= 0,None))",Success,
7,Review Check,Warning,Success,"UniquenessConstraint(Uniqueness(List(ProductID),None))",Success,
8,Review Check,Warning,Success,"ComplianceConstraint(Compliance(Style contained in U ,W ,M ,`Style` IS NULL OR `Style` IN ('U ','W ','M '),None))",Success,
9,Review Check,Warning,Success,"ComplianceConstraint(Compliance(Color contained in Black,Silver,Red,Yellow,Blue,Multi,Silver/Black,White,Grey,Brown,`Color` IS NULL OR `Color` IN ('Black','Silver','Red','Yellow','Blue','Multi','Silver/Black','White','Grey','Brown'),None))",Success,


In [11]:
import os

with open("constraint_verification_report.html", "w") as tr:
    tr.write(html_report)